In [2]:
import pandas as pd
import spacy
from spacy.tokens import Doc
from spacytextblob.spacytextblob import SpacyTextBlob
from tabulate import tabulate

In [3]:
all_data = pd.read_csv(r"C:\Users\abhyu\Desktop\Dissertation\DATA\Important Data\Two K\twoClusterCategorised_allData.csv")
# all_data.columns
filtered_all_data = all_data.dropna(subset=['review'])
# filtered_all_data
set(filtered_all_data.loc[filtered_all_data['cluster']==1]['username'].to_list())

filtered_all_data

,username,movie_id,title,year,length,review,user_rating,letterboxd_views,letterboxd_rating,imdb_rating,...,budget,gross,aspect_ratio,letterboxd_link,imdb_link,tmdb_link,user_type,language_diversity_index,decade_diversity_index,cluster
0,grantmulligan,444600,Jojo Rabbit,2019.0,108.0,So close to being great. Just a bit too satiri...,7.0,1324817,8.12,7.9,...,14000000.0,93757609.0,Aspect ratio1.85 : 1,https://letterboxd.com/film/jojo-rabbit/,http://www.imdb.com/title/tt2584384/maindetails,https://www.themoviedb.org/movie/515001/,reg_user,0.0,0.6176,0
1,grantmulligan,51815,Natural Born Killers,1994.0,118.0,You can just tell this was made by a wannabe T...,6.0,231925,7.08,7.2,...,34000000.0,50283563.0,Aspect ratio1.85 : 1,https://letterboxd.com/film/natural-born-killers/,http://www.imdb.com/title/tt0110632/maindetails,https://www.themoviedb.org/movie/241/,reg_user,0.0,0.6176,0
2,grantmulligan,51521,Full Metal Jacket,1987.0,117.0,Understand why ppl have it as the goat war fil...,9.0,647543,8.26,8.3,...,30000000.0,46358827.0,NaN,https://letterboxd.com/film/full-metal-jacket/,http://www.imdb.com/title/tt0093058/maindetails,https://www.themoviedb.org/movie/600/,reg_user,0.0,0.6176,0
3,grantmulligan,51613,The Green Mile,1999.0,189.0,Stephen King is the definition of having to se...,8.0,655388,8.28,8.6,...,60000000.0,286801374.0,Aspect ratio1.85 : 1,https://letterboxd.com/film/the-green-mile/,http://www.imdb.com/title/tt0120689/maindetails,https://www.themoviedb.org/movie/497/,reg_user,0.0,0.6176,0
4,grantmulligan,71338,The Perks of Being a Wallflower,2012.0,103.0,Moral dilemma on what is the appropriate age o...,8.0,1343186,7.94,7.9,...,13000000.0,33384127.0,Aspect ratio1.85 : 1,https://letterboxd.com/film/the-perks-of-being...,http://www.imdb.com/title/tt1659337/maindetails,https://www.themoviedb.org/movie/84892/,reg_user,0.0,0.6176,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36995,iamthatroby,563017,Shithouse,2020.0,102.0,Unbelievably sweet movie! Insane that this is ...,8.0,46143,7.52,6.9,...,NaN,18370.0,Aspect ratio1.85 : 1,https://letterboxd.com/film/shithouse/,http://www.imdb.com/title/tt11618536/maindetails,https://www.themoviedb.org/movie/637053/,pop_user,0.0,0.6904,0
36996,iamthatroby,333029,Call Me by Your Name,2017.0,132.0,luca put his whole guadanussy into this one.,10.0,1370660,7.86,7.8,...,4000000.0,43143046.0,Aspect ratio1.85 : 1,https://letterboxd.com/film/call-me-by-your-name/,http://www.imdb.com/title/tt5726616/maindetails,https://www.themoviedb.org/movie/398818/,pop_user,0.0,0.6904,0
36997,iamthatroby,33714,Kicking and Screaming,1995.0,96.0,pretty bad but it’s very inspiring to see what...,7.0,47934,6.66,6.7,...,NaN,718490.0,Aspect ratio1.85 : 1,https://letterboxd.com/film/kicking-and-scream...,http://www.imdb.com/title/tt0113537/maindetails,https://www.themoviedb.org/movie/28387/,pop_user,0.0,0.6904,0
36998,iamthatroby,43810,Empire Records,1995.0,90.0,robin tunney an all timer I fear,5.0,111269,7.06,6.7,...,NaN,273188.0,Aspect ratio2.39 : 12.35 : 1,https://letterboxd.com/film/empire-records/,http://www.imdb.com/title/tt0112950/maindetails,https://www.themoviedb.org/movie/13531/,pop_user,0.0,0.6904,0


In [245]:
review = 0
nlp = spacy.load('en_core_web_sm')
# Define custom stop words list
custom_stop_words = nlp.Defaults.stop_words - set(["no", "not", "never", "none", "neither", "nor", "nobody", "nothing", "nowhere", "hardly", "scarcely", "barely"])

# Set custom stop words list in SpaCy
nlp.Defaults.stop_words = custom_stop_words

nlp.add_pipe('spacytextblob')

In [246]:
def spacy_sentimental_analysis(review):
    doc = nlp(review)
    polarity = doc._.blob.polarity                           
    subjectivity = doc._.blob.subjectivity                        
    return [polarity, subjectivity]

def preprocess(text):
    text = text.replace('\n', ' ')
    text = text.replace('\xa0', ' ')
    doc = nlp(text)
    tokens = [token.lemma_.lower() for token in doc if not token.is_punct and not token.is_stop]
    return ' '.join(tokens)

def spacy_review_topics(review, movie_review_dictionary):

    text = preprocess(review)
    doc = nlp(text)
    review_length = len(text.split())
    # Check if any of the topic words are in the review
    story_mentioned = 0
    filmmaking_mentioned = 0
    adjectives_mentioned = 0
    for token in doc:
        if token.text.lower() in set(movie_review_dictionary['story_vocabulary']):
            story_mentioned = 1
        if token.text.lower() in set(movie_review_dictionary['filmmaking_review_vocabulary']):
            filmmaking_mentioned = 1
        if token.text.lower() in set(movie_review_dictionary['movie_review_adjectives']):
            adjectives_mentioned = 1
    return [story_mentioned, filmmaking_mentioned, adjectives_mentioned, review_length]

In [2]:
#DICTIONARY
movie_review_dictionary = {
    "story_vocabulary": [
        "exposition", "villain", "dialogue", "inciting", "climax", "resolution", "denouement",
        "twist", "foreshadowing", "flashback", "prologue", "epilogue", "subplot", "parallel",
        "episodic", "circular", "question", "conflict", "internal", "external", "society",
        "supernatural", "theme", "symbolism", "mood", "tone", "narrator", "point", "first",
        "second", "third", "unreliable", "characterization", "protagonist", "antagonist",
        "foil", "dynamic", "static", "round", "flat", "stock", "irony", "verbal", "situational",
        "dramatic", "metaphor", "simile", "personification", "hyperbole", "allusion", "imagery",
        "setting", "atmosphere", "world-building", "redemption", "rising action", "falling action",
        "climax reversal", "twist ending", "subtext", "motivation", "back-story", "pacing",
        "tension", "stakes", "pacing", "pacing", "irony", "tragedy", "catharsis", "pathos",
        "epiphany", "character arc", "anti-hero", "mentor", "archetypes", "bildungsroman"
    ],

    "filmmaking_review_vocabulary": [
        "cinematography", "composition", "lighting", "performance", "acting", "colour", "shot",
        "frame", "stabilization", "focus", "depth of field", "aspect ratio", "editing", "pace",
        "montage", "transition", "sound", "music", "foley", "designsound effects", "mixing",
        "mastering", "visual effects", "animation", "green screen", "practical effects", "makeup",
        "costume", "set decoration", "location", "props", "stunts", "camera", "electric",
        "production", "color", "camera movement", "dolly shot", "handheld", "crane shot",
        "tracking shot", "zoom", "close-up",
        "low angle", "high angle", "dutch angle","grading", "sound mix", "visual storytelling"
    ],
    "movie_review_adjectives" : ['riveting', 'engrossing', 'compelling', 'gripping', 'intense', 'captivating', 'thrilling', 'exciting', 'exhilarating', 'heartwarming', 'touching', 'emotional', 'poignant', 'thought-provoking', 'insightful', 'profound', 'impactful', 'challenging', 'contemplative', 'haunting', 'atmospheric', 'immersive', 'cinematic', 'stunning', 'breathtaking', 'impressive', 'beautiful', 'gorgeous', 'spectacular', 'explosive', 'action-packed', 'funny', 'hilarious', 'witty', 'clever', 'smart', 'surprising', 'unpredictable', 'original', 'innovative', 'bold', 'daring', 'groundbreaking', 'timely', 'relevant', 'powerful', 'suspenseful', 'mysterious', 'intriguing', 'complicated', 'convoluted', 'confusing', 'overwhelming', 'long', 'dragging', 'boring', 'uninspired', 'forgettable', 'disappointing', 'underwhelming', 'cliché', 'derivative', 'predictable', 'formulaic', 'generic', 'mediocre', 'bad', 'terrible', 'awful']
}




# movie_review_dictionary = {
#     "story_vocabulary":  [    "exposition",   "dialogue",     "inciting"],

#     "filmmaking_review_vocabulary": ['cinematography', 'composition', 'lighting'],

#     "movie_review_adjectives": ['riveting', 'engrossing', 'compelling']
# }

In [5]:
print(movie_review_dictionary['movie_review_adjectives'])

['riveting', 'engrossing', 'compelling', 'gripping', 'intense', 'captivating', 'thrilling', 'exciting', 'exhilarating', 'heartwarming', 'touching', 'emotional', 'poignant', 'thought-provoking', 'insightful', 'profound', 'impactful', 'challenging', 'contemplative', 'haunting', 'atmospheric', 'immersive', 'cinematic', 'stunning', 'breathtaking', 'impressive', 'beautiful', 'gorgeous', 'spectacular', 'explosive', 'action-packed', 'funny', 'hilarious', 'witty', 'clever', 'smart', 'surprising', 'unpredictable', 'original', 'innovative', 'bold', 'daring', 'groundbreaking', 'timely', 'relevant', 'powerful', 'suspenseful', 'mysterious', 'intriguing', 'complicated', 'convoluted', 'confusing', 'overwhelming', 'long', 'dragging', 'boring', 'uninspired', 'forgettable', 'disappointing', 'underwhelming', 'cliché', 'derivative', 'predictable', 'formulaic', 'generic', 'mediocre', 'bad', 'terrible', 'awful']


In [248]:
def review_analysis(review, movie_review_dictionary):
    
    sentiment = spacy_sentimental_analysis(review)
    review_topics = spacy_review_topics(review, movie_review_dictionary)
    attributes_list = [review] + sentiment + review_topics
    review_df = pd.DataFrame(attributes_list).T
    # print(review_df)
    review_df.columns = ['review', 'polarity', 'subjectivity', 'story_mentioned', 'filmmaking_mentioned', 'adjectives_mentioned', 'review_word_count']
    return review_df

In [249]:
df = pd.DataFrame(columns=['review', 'polarity', 'subjectivity', 'story_mentioned', 'filmmaking_mentioned', 'adjectives_mentioned', 'review_word_count'])
# print(df)
i=0
for review in filtered_all_data['review']:
    review_df = review_analysis(review, movie_review_dictionary)
    df = pd.concat([df, review_df])
    # print(df)
    # print(review_df)
    i+=1
    if i % 100==0:
        print(i)
# print(tabulate(df, headers='keys', tablefmt='psql'))

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [250]:
df

,review,polarity,subjectivity,story_mentioned,filmmaking_mentioned,adjectives_mentioned,review_word_count
0,So close to being great. Just a bit too satiri...,0.578571,0.642857,0,0,0,8
0,You can just tell this was made by a wannabe T...,0.0,0.0,0,0,0,3
0,Understand why ppl have it as the goat war fil...,0.0,0.375,0,0,0,9
0,Stephen King is the definition of having to se...,0.0,0.0,1,0,0,11
0,Moral dilemma on what is the appropriate age o...,0.25,0.533333,0,0,0,8
...,...,...,...,...,...,...,...
0,Unbelievably sweet movie! Insane that this is ...,0.164583,0.522222,0,0,0,13
0,luca put his whole guadanussy into this one.,0.2,0.4,0,0,0,2
0,pretty bad but it’s very inspiring to see what...,0.066667,0.888889,0,0,1,7
0,robin tunney an all timer I fear,0.0,0.0,0,0,0,4


In [252]:
new_all_data = pd.merge(all_data, df, on='review', how='outer', suffixes=('', '_drop'))
new_all_data.drop([col for col in new_all_data.columns if col.endswith('_drop')], axis=1, inplace=True)
new_all_data

,username,movie_id,title,year,length,review,user_rating,letterboxd_views,letterboxd_rating,imdb_rating,...,user_type,language_diversity_index,decade_diversity_index,cluster,polarity,subjectivity,story_mentioned,filmmaking_mentioned,adjectives_mentioned,review_word_count
0,grantmulligan,444600,Jojo Rabbit,2019.0,108.0,So close to being great. Just a bit too satiri...,7.0,1324817,8.12,7.9,...,reg_user,0.0,0.6176,0,0.578571,0.642857,0,0,0,8
1,grantmulligan,51815,Natural Born Killers,1994.0,118.0,You can just tell this was made by a wannabe T...,6.0,231925,7.08,7.2,...,reg_user,0.0,0.6176,0,0.0,0.0,0,0,0,3
2,grantmulligan,51521,Full Metal Jacket,1987.0,117.0,Understand why ppl have it as the goat war fil...,9.0,647543,8.26,8.3,...,reg_user,0.0,0.6176,0,0.0,0.375,0,0,0,9
3,grantmulligan,51613,The Green Mile,1999.0,189.0,Stephen King is the definition of having to se...,8.0,655388,8.28,8.6,...,reg_user,0.0,0.6176,0,0.0,0.0,1,0,0,11
4,grantmulligan,71338,The Perks of Being a Wallflower,2012.0,103.0,Moral dilemma on what is the appropriate age o...,8.0,1343186,7.94,7.9,...,reg_user,0.0,0.6176,0,0.25,0.533333,0,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39519,iamthatroby,563017,Shithouse,2020.0,102.0,Unbelievably sweet movie! Insane that this is ...,8.0,46143,7.52,6.9,...,pop_user,0.0,0.6904,0,0.164583,0.522222,0,0,0,13
39520,iamthatroby,333029,Call Me by Your Name,2017.0,132.0,luca put his whole guadanussy into this one.,10.0,1370660,7.86,7.8,...,pop_user,0.0,0.6904,0,0.2,0.4,0,0,0,2
39521,iamthatroby,33714,Kicking and Screaming,1995.0,96.0,pretty bad but it’s very inspiring to see what...,7.0,47934,6.66,6.7,...,pop_user,0.0,0.6904,0,0.066667,0.888889,0,0,1,7
39522,iamthatroby,43810,Empire Records,1995.0,90.0,robin tunney an all timer I fear,5.0,111269,7.06,6.7,...,pop_user,0.0,0.6904,0,0.0,0.0,0,0,0,4


In [ ]:
# new_all_data.to_csv(r"C:\Users\abhyu\Desktop\Dissertation\DATA\Important Data\Two K\NLP_allData.csv", index=False)

In [7]:
user_df = pd.read_csv(r"C:\Users\abhyu\Desktop\Dissertation\DATA\Important Data\Two K\twoClusterCategorisedUsers.csv")
all_data = pd.read_csv(r"C:\Users\abhyu\Desktop\Dissertation\DATA\Important Data\Two K\NLP_allData.csv")
filtered_all_data = all_data.dropna(subset=['review'])

usernames = user_df['username'].to_list()
len(usernames)
# user_df

736

In [16]:
final_mean_values_df= pd.DataFrame(columns=[
    'username',
    'mean_polarity',
    'mean_subjectivity',
    'mean_story_mentioned',
    'mean_filmmaking_mentioned',
    'mean_adjectives_mentioned',
    'mean_review_word_count'])
for user in usernames:
    df = filtered_all_data[filtered_all_data['username']==user]
    # 'polarity', 'subjectivity',
    #    'story_mentioned', 'filmmaking_mentioned', 'adjectives_mentioned',
    #    'review_word_count'
    mean_polarity = df['polarity'].mean()
    mean_subjectivity = df['subjectivity'].mean()
    mean_story_mentioned = df['story_mentioned'].mean()
    mean_filmmaking_mentioned = df['filmmaking_mentioned'].mean()
    mean_adjectives_mentioned = df['adjectives_mentioned'].mean()
    mean_review_word_count = df['review_word_count'].mean()

    mean_values = [
    user,
    mean_polarity,
    mean_subjectivity,
    mean_story_mentioned,
    mean_filmmaking_mentioned,
    mean_adjectives_mentioned,
    mean_review_word_count]
    mean_values_df= pd.DataFrame(mean_values).T
    mean_values_df.columns = [
    'username',
    'mean_polarity',
    'mean_subjectivity',
    'mean_story_mentioned',
    'mean_filmmaking_mentioned',
    'mean_adjectives_mentioned',
    'mean_review_word_count']
    final_mean_values_df = pd.concat([final_mean_values_df, mean_values_df])
    # df = pd.concat([df, review_df])

    # print(final_mean_values_df)
    # break

In [17]:
final_mean_values_df

,username,mean_polarity,mean_subjectivity,mean_story_mentioned,mean_filmmaking_mentioned,mean_adjectives_mentioned,mean_review_word_count
0,orpheus1,0.150844,0.421877,0.040816,0.061224,0.020408,8.44898
0,movies4mejason,0.201875,0.474353,0.039216,0.039216,0.137255,5.196078
0,youthihara,0.047131,0.227512,0.0,0.04,0.06,31.16
0,bergmanlynch,0.02513,0.33193,0.06,0.06,0.1,164.84
0,gargen,-0.142726,0.308913,0.0,0.142857,0.142857,197.857143
...,...,...,...,...,...,...,...
0,vegto389,0.08026,0.510339,0.25,0.0625,0.375,31.625
0,davert22,0.068851,0.455652,0.28,0.28,0.34,49.88
0,raimibaby,0.102276,0.567697,0.076923,0.0,0.230769,16.692308
0,mstewart9753,-0.012824,0.345741,0.0,0.044444,0.022222,5.4


In [18]:
new_user_data = pd.merge(user_df, final_mean_values_df, on='username', how='outer', suffixes=('', '_drop'))
new_user_data.drop([col for col in new_user_data.columns if col.endswith('_drop')], axis=1, inplace=True)
new_user_data

,username,language_diversity_index,decade_diversity_index,user_type,cluster,mean_polarity,mean_subjectivity,mean_story_mentioned,mean_filmmaking_mentioned,mean_adjectives_mentioned,mean_review_word_count
0,orpheus1,0.3200,0.648000,reg_user,1,0.150844,0.421877,0.040816,0.061224,0.020408,8.44898
1,movies4mejason,0.2560,0.776000,reg_user,1,0.201875,0.474353,0.039216,0.039216,0.137255,5.196078
2,youthihara,0.6040,0.753600,reg_user,1,0.047131,0.227512,0.0,0.04,0.06,31.16
3,bergmanlynch,0.4584,0.826400,reg_user,1,0.02513,0.33193,0.06,0.06,0.1,164.84
4,gargen,0.7312,0.751200,reg_user,1,-0.142726,0.308913,0.0,0.142857,0.142857,197.857143
...,...,...,...,...,...,...,...,...,...,...,...
731,vegto389,0.0776,0.662400,pop_user,0,0.08026,0.510339,0.25,0.0625,0.375,31.625
732,davert22,0.1152,0.644000,pop_user,0,0.068851,0.455652,0.28,0.28,0.34,49.88
733,raimibaby,0.1856,0.839200,pop_user,0,0.102276,0.567697,0.076923,0.0,0.230769,16.692308
734,mstewart9753,0.0776,0.414827,pop_user,0,-0.012824,0.345741,0.0,0.044444,0.022222,5.4


In [ ]:
new_user_data.to_csv(r"C:\Users\abhyu\Desktop\Dissertation\DATA\Important Data\Two K\complete_user_data.csv", index=False)